# 1.获取小说资源
    自行下载epub文件，在此转换为txt格式并合并为一个文件,已有txt格式则忽略这步

In [58]:
# from ebooklib import epub
# from bs4 import BeautifulSoup
# import os

# def extract_text_from_epub(file_path):
#     book = epub.read_epub(file_path)
#     text_content = []
#     for item in book.items:
#         if item.media_type == 'application/xhtml+xml':
#             soup = BeautifulSoup(item.content,'html.parser')
#             text_content.append(soup.get_text())
#     return "\n".join(text_content)

# def merge_epubs_to_txt(epub_files, output_txt_file):
#     with open(output_txt_file, "w", encoding="utf-8") as f:
#         for file_name in os.listdir(epub_files):
#             if file_name.endswith('.epub'):
#                 file_path = os.path.join(epub_files,file_name)
#                 print(f"Porcess {file_name}...")
#                 text = extract_text_from_epub(file_path)
#                 f.write(text + "\n\n")


# # 使用示例
# epub_files = "./novel/败犬1-6-epub"
# output_txt_file = "TooManyLosingHeroines.txt"   
# merge_epubs_to_txt(epub_files, output_txt_file)

# 2.小说切分

In [59]:
# 选择你要处理的小说，接下来的保存路径都将使用小说名
novel_name = "TooManyLosingHeroines"
input_txt_name = f"./novel/{novel_name}.txt"
save_folder = f"./extract/{novel_name}"

In [60]:
import os

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
else:
    print('注意，文件夹',save_folder,'已经存在')

raw_text = open(input_txt_name, encoding='utf-8').read()

# 将文本按章节切分
chapters = []
chapter_contents = []

for line in raw_text.split('\n'):
    Flag = False
    if line.strip().startswith("～第"):
        # 遇到章节标题,将之前章节内容添加到结果列表

        head = line.strip()
        # print(head)
        head = head[:min(10,len(head))]
        if head.find('败',1)>0:
            # print(head)
            Flag = True

    if Flag:
        if chapter_contents:
            chapters.append('\n'.join(chapter_contents))
            chapter_contents = []
        # 记录当前章节标题
        # chapters.append(line)
    else:
        # 累积章节内容
        chapter_contents.append(line)

# 添加最后一个章节内容
if chapter_contents:
    chapters.append('\n'.join(chapter_contents))
# 检查结果
print(len(chapters))
for i,ch in enumerate(chapters):
    print(i,ch[:10])

注意，文件夹 ./extract/TooManyLosingHeroines 已经存在
49
0 








败
1 
    专业青梅竹
2 
    为你献上注
3 
    未战先输　
4 
    当你凝望败
5 『哥哥真的很努力了喔
6 蝉声如雨。
隔天的第
7 隔天，合宿第一天。放
8 合宿的隔天。星期一早
9 八奈见杏菜想暗示
I
10 朝云千早令人混乱
11 
    唯独没被甩
12 烧盐柠檬开了口
尾声
13 「我回来了……」
我
14 与朝云千早约定好提供
15 这一天的上午。太阳已
16 隔天夜里。我在精文馆
17 志喜屋梦子多方关照

18 姬宫华恋隆重登场
I
19 
    要说再见还
20 
    来讨论责任
21 西校舍的角落，文艺社
22 文艺部的顾问老师决定
23 石蕗祭当天早晨，我正
24 石蕗祭结束后过了三天
25 
    别看我这样
26 
    只要多一点
27 
    为我的心取
28 
    十六岁的序
29 放学后的社办中，摆在
30 星期五课程结束后的班
31 12月21日，星期一
32 12月25日早晨。阳
33 
    道高一尺，
34 
    以浑身解数
35 
    欲擒主将，
36 
    放手的觉悟
37 期末考第四天结束，就
38 周末过去，星期一到学
39 隔天的放学后。我来到
40 石蕗高中参观会当天。
41 
    海风的告白
42 
    流泪愈多，
43 
    道别的季节
44 
    烧盐柠檬这
45 石蕗高中文艺社的社办
46 周末结束后的星期一总
47 万里无云的星期六午后
48 八奈见发出敌对宣言后


In [61]:
# @title  定义divide函数，用来切分超长文本
def divide_str(s, sep=["\n", ".", "。"]):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, 0, mid_len)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos - mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ""
    return s[: best_sep_pos + 1], s[best_sep_pos + 1 :]


def strong_divide(s):
    left, right = divide_str(s)

    if right != "":
        return left, right

    whole_sep = [
        "\n",
        ".",
        "，",
        "、",
        ";",
        ",",
        "；",
        "：",
        "！",
        "？",
        "(",
        ")",
        "”",
        "“",
        "’",
        "‘",
        "[",
        "]",
        "{",
        "}",
        "<",
        ">",
        "/",
        """''', '|', '-', '=', '+', '*', '%', \
               '$', """  #''', '@', '&', '^', '_', '`', '~',\
        "·",
        "…",
    ]
    left, right = divide_str(s, sep=whole_sep)

    if right != "":
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]

In [62]:
# @title 以1500 token为限，切分chunk，输出总chunk数量
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
max_token_len = 1500
chunk_text = []


for chapter in chapters:

    split_text = chapter.split("\n")

    curr_len = 0
    curr_chunk = ""

    tmp = []

    for line in split_text:
        line_len = len(enc.encode(line))

        if line_len <= max_token_len - 5:
            tmp.append(line)
        else:
            # print('divide line with length = ', line_len)
            path = [line]
            tmp_res = []

            while path:
                my_str = path.pop()
                left, right = strong_divide(my_str)

                len_left = len(enc.encode(left))
                len_right = len(enc.encode(right))

                if len_left > max_token_len - 15:
                    path.append(left)
                else:
                    tmp_res.append(left)

                if len_right > max_token_len - 15:
                    path.append(right)
                else:
                    tmp_res.append(right)

            for line in tmp_res:
                tmp.append(line)

    split_text = tmp

    for line in split_text:
        line_len = len(enc.encode(line))

        if line_len > max_token_len:
            print("warning line_len = ", line_len)

        if curr_len + line_len <= max_token_len:
            curr_chunk += line
            curr_chunk += "\n"
            curr_len += line_len
            curr_len += 1
        else:
            chunk_text.append(curr_chunk)
            curr_chunk = line
            curr_len = line_len

    if curr_chunk:
        chunk_text.append(curr_chunk)

    # break

print(len(chunk_text))
print(chunk_text[20])

582
就在这时，突然走进教室的男人笔直地走向我的座位。「呐，温水。听说你加入文艺社了？」
「呃……」
向我搭话的是D班的绫野光希。我们是同一所国中出身，算是朋友──其实只是以前上同一所补习班，他偶尔会找我搭话，这种程度的交情而已。
顺带一提，我和他只是上同一间补习班，成绩是那家伙好过我许多。毕竟他还戴着眼镜。
「呃，是啊。算是这样没错。」
「我听老师说那边有安部公房的全集，下次可以去借书吗？」
哦～原来还有这种书喔。我只注意过轻小说的藏书种类而已。
「这个，应该没关系吧。我去找学长姐拜托看看。」
「不好意思，谢啦。」
绫野面露爽朗的笑容，拍打我的肩膀后，随即准备离开此处。
就在这时，小麦色泽的身影冲进视野中。
现身的是烧盐柠檬。那双晒黑的手臂压在桌面上。
「光希！先等一下。」
烧盐的身子朝着绫野前倾，8×4与汗水彼此混合的气味顿时飘荡在空气中。



译注：制汗喷雾。


……太近了。而且很碍事。
「我今天田径队不用练习喔，要不要一起去吃点东西？」
「抱歉，今天要上补习班。」
绫野双手合十，摆出「抱歉」的小动作。
「咦～不是才一年级吗？老是念书会变成笨蛋喔。」
「你倒是该多念点书。不然会被留级喔。」
……这两个家伙，居然在我的座位旁边开始打情骂俏了。
「光希同学，再不快点出发，会赶不上补习班喔。」
教室的入口处，一位身材苗条的女学生探出头来。
咦？我记得这个女生，就是在补习班常常和绫野出双入对的女生。人长得可爱、成绩又很好，在补习班相当知名，原来我们同校喔。
「啊啊，千早。我马上来。那我先走啦，柠檬。」
「欸……嗯，掰掰……」
毫不掩饰心中失落，烧盐消沉地挥了挥手。
感觉好像很麻烦，我想早点离开，不过因为被烧盐挡住，我没办法拿书包。
「那、那个……烧盐同学……那个……书包……」
「呐，原来温水和光希是朋友喔？你们不同班吧？」
烧盐疑惑地眨着眼，两排纤长的睫毛也随之颤动，她直盯着我的脸看。
──田径队的短跑主将，烧盐柠檬。在班上是相当引人注目的女学生。
剪短的头发包覆着小巧的脸蛋，从制服伸出的四肢苗条而精实，被晒成了好看的小麦色。我一瞬间看呆，佯装平静地开口说道：
「呃……也算不上朋友。只是之前上同一间补习班，偶尔会讲几句话。」
烧盐突然间眼神绽放光芒。
「是同一间补习班喔！那你认识刚才那个女生吗！？」
烧盐情绪激动，脸庞直逼向我眼前，这下我也难掩惊慌。
「她喔，我

# 3.重组小说
    将小说重组为人物对话和剧情摘要

In [63]:
!pip install langchain_community kor
!pip install -U langchain langchain-openai

In [64]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
import os

# 设置你自己的LLM key与model
key = os.getenv("OPENAI_API_KEY", "your_api_key")
model = "gpt-4o"

llm = ChatOpenAI(model_name=model, temperature=0)

In [65]:
# @title 仅仅抽取dialogue的部分

# @title 抽取role,dialogue,action

schema = Object(
    id="script",
    description='''Adapted from the novel into script,
    The main character,“温水”, refers to himself as “我” ''',
    attributes=[
        Text(
            id="role",
            description="The character who is speaking, use context to predict the name of the role.",
        ),
        Text(
            id="dialogue",
            description="The dialogue spoken by the characters in the sentence",
        ),
        Text(
            id ="action",
            descrtption = '''The actions performed by the characters in the text, A high-level summary of a character's behavior. action equals "对话" or "独白" or "动作", 
            equals "对话" if it's sentence in [], equals "独白" elif it's a psychological monologue not in [], equals "动作" else
            ''',
        ),
    ],
    examples=[
        (
            """``八奈见不知何时来到我身旁，用手肘轻轻地顶我。
「喔，烧盐同学好像喜欢绫野的样子。」
「是喔。真是意外的组合呢。」
「我和他们国中时同校，不过他们好像从国小就常常在一起。」
「所以说，就和青梅竹马差不多啰。」
青梅竹马……在八奈见眼中是这样吗？
对着无法理解的我，八奈见无奈地耸了耸肩。
「听我说，温水，女生可以分成两大类。如果不是青梅竹马，就是狐狸精。」
原来如此，真豪爽的分类法。八奈见用严厉的表情看着我。``""",
            [
                {"role":"八奈见","dialogue":"八奈见不知何时来到我身旁，用手肘轻轻地顶我","action":"动作",},
                {"role": "八奈见", "dialogue": "喔，烧盐同学好像喜欢绫野的样子","action":"对话",},
                {"role": "温水", "dialogue": "是喔。真是意外的组合呢。","action":"对话",},
                {
                    "role": "温水",
                    "dialogue": "我和他们国中时同校，不过他们好像从国小就常常在一起",
                    "action":"对话",
                },
                {
                    "role": "温水",
                    "dialogue": "青梅竹马……在八奈见眼中是这样吗？",
                    "action":"独白",
                },
                {"role": "八奈见", "dialogue": "所以说，就和青梅竹马差不多啰","action":"对话",},
                {
                    "role": "八奈见",
                    "dialogue": "听我说，温水，女生可以分成两大类。如果不是青梅竹马，就是狐狸精",
                    "action":"对话",
                },
                {
                    "role": "温水",
                    "dialogue": "原来如此，真豪爽的分类法",
                    "action":"独白",
                },
            ],
        )
    ],
    many=True,
)
chain = create_extraction_chain(llm, schema)

In [66]:
# test
test_chunk_id = 300
response = chain.invoke({"text":f"``{chunk_text[test_chunk_id]}"})
print(response["data"])

{'script': [{'role': '志喜屋', 'dialogue': '别紧张……天爱星……很好搞定……', 'action': '志喜屋学姐不知为何用双手指头圈成爱心形状'}, {'role': '温水', 'dialogue': '就算是这样，要我去攻陷马剃同学，真的办不到啦。', 'action': '对话'}, {'role': '八奈见', 'dialogue': '温水想太多了啦。就只是和马剃同学打好关系，拜托她把同人志还给我们，这样而已嘛。对吧？学姐。', 'action': '对话'}, {'role': '温水', 'dialogue': '是这样的话，不管怎么想，还是八奈见同学去做比较容易吧？', 'action': '对话'}, {'role': '八奈见', 'dialogue': '那女生有点可怕，我不要。', 'action': '对话'}, {'role': '温水', 'dialogue': '学姐，我真的不觉得有办法和马剃同学打好关系耶。', 'action': '对话'}, {'role': '志喜屋', 'dialogue': '天爱星……只要温柔对待……很简单……', 'action': '对话'}, {'role': '温水', 'dialogue': '呃，可是──', 'action': '对话'}, {'role': '志喜屋', 'dialogue': '抓住把柄……卖人情……用罪恶感……堵住去路。', 'action': '志喜屋学姐低语，将小小的黑色棋子摆到盘面上'}, {'role': '志喜屋', 'dialogue': '我会……帮忙喔？', 'action': '对话'}, {'role': '温水', 'dialogue': '哎，算是啦。', 'action': '对话'}, {'role': '八奈见', 'dialogue': '既然学姐都说愿意帮忙了，你就做好觉悟吧。', 'action': '对话'}, {'role': '温水', 'dialogue': '但是说到底，我很不擅长应付女生啊。只要一面对面就会忘记怎么说话。', 'action': '对话'}, {'role': '八奈见', 'dialogue': '温水，你要不要偶尔想起来我是女的？', 'action': '八奈

In [67]:
system_prompt = """
Summarize the key points of the following text in a concise way, using bullet points.
"""

q_example = """###
Text:
洪七公、周伯通、郭靖、黄蓉四人乘了小船，向西驶往陆地。郭靖坐在船尾扳桨，黄蓉不住向周伯通详问骑鲨游海之事，周伯通兴起，当场就要设法捕捉鲨鱼，与黄蓉大玩一场。
郭靖见师父脸色不对，问道：“你老人家觉得怎样”洪七公不答，气喘连连，声息粗重。他被欧阳锋以“透骨打穴法”点中之后，穴道虽已解开，内伤却又加深了一层。黄蓉喂他服了几颗九花玉露丸，痛楚稍减，气喘仍是甚急。
老顽童不顾别人死活，仍是嚷着要下海捉鱼，黄蓉却已知不妥，向他连使眼色，要他安安静静的，别吵得洪七公心烦。周伯通并不理会，只闹个不休。黄蓉皱眉道：“你要捉鲨鱼，又没饵引得鱼来，吵些甚么”

Summarize in BULLET POINTS form:
"""

a_example = """
- 洪七公等四人乘船西行,洪七公因受内伤加重而气喘不止
- 周伯通要捉鲨鱼玩,被黄蓉阻止以免掀翻小船
"""

In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [69]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=q_example),
    AIMessage(content=a_example),
]


new_input = f"""###
Text:
{chunk_text[test_chunk_id]}

Summarize in BULLET POINTS form:"""

messages.append(HumanMessage(content=new_input))

response = llm(messages)

print(response.content)


- 主角被志喜屋学姐鼓励去与马剃同学打好关系以取回同人志
- 主角对与女生打交道感到不安，尤其是马剃同学
- 志喜屋学姐认为马剃同学容易对付，并愿意帮忙
- 主角担心马剃同学对文艺社有敌意，需谨慎处理
- 八奈见提醒主角不要误解任务为勾引
- 主角感到疲惫，回家后在沙发上休息


In [70]:
import os
import json
from tqdm import tqdm

# 最贵的一步，这里只处理几个chunk作为测试
for i in tqdm(range(len(chunk_text))):
    save_name = os.path.join(save_folder, f"{i}_dialogue.txt")

    if not os.path.exists(save_name) or os.path.getsize(save_name) < 5:
        if os.path.exists(save_name):
            print("re-generate dialogue id = ", i)
        query_text = f"``{chunk_text[i]}``"
        # dialogue_response = chain.run( query_text )["data"]
        dialogue_response = chain.invoke({"text": f"``{chunk_text[i]}``"})["data"]
        with open(save_name, "w", encoding="utf-8") as f:
            if "script" not in dialogue_response:
                print('Error: response does not contain key "script"')
            else:
                for chat in dialogue_response["script"]:
                    json_str = json.dumps(chat, ensure_ascii=False)
                    f.write(json_str + "\n")

    save_name_sum = os.path.join(save_folder, f"{i}_sum.txt")

    if not os.path.exists(save_name_sum) or os.path.getsize(save_name_sum) < 5:
        if os.path.exists(save_name_sum):
            print("re-summarize id = ", i)
        # dealing with summarize
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=q_example),
            AIMessage(content=a_example),
        ]

        new_input = f"###\nText:\n{chunk_text[ i ]}\nSummarize in BULLET POINTS form:"

        messages.append(HumanMessage(content=new_input))

        summarize_response = llm(messages).content

        with open(save_name_sum, "w", encoding="utf-8") as f:
            f.write(summarize_response)

    raw_text_save_name = os.path.join(save_folder, f"{i}_raw.txt")

    if (
        not os.path.exists(raw_text_save_name)
        or os.path.getsize(raw_text_save_name) < 5
    ):
        with open(raw_text_save_name, "w", encoding="utf-8") as f:
            f.write(chunk_text[i])
    # # for test
    # if i > 3:
    #     break

100%|██████████| 582/582 [3:00:28<00:00, 18.61s/it]    


## 将得到的raw,dialogue,sum组合为规格的json

In [71]:
save_folder_path = f"./reorganized/{novel_name}"
# chunk所在文件夹，请以_raw结尾
folder_path = f"./extract/{novel_name}"
# 故事名字，默认为_raw之前的名字
story_name_en = novel_name

save_jsonl_path = f"{save_folder_path}/reorganized_{story_name_en}.jsonl"
save_txt_path = f"{save_folder_path}/reorganized_{story_name_en}.txt"


In [72]:
chunk_text = [""] * (
    sum([1 for file_name in os.listdir(folder_path) if file_name.endswith("_sum.txt")])
)

# 遍历文件夹中的文件
for file_name in os.listdir(folder_path):
    if file_name.endswith("_raw.txt"):
        file_path = os.path.join(folder_path, file_name)

        # 提取文件名中的 i
        i = int(file_name.split("_")[0])

        # 打开文件并读取内容
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()

        # 将文件内容添加到列表中，按 i 的顺序插入到正确位置
        chunk_text[i] = text

print(len(chunk_text))

582


In [74]:

id = 2
dialoge_file = os.path.join(folder_path, f"{id}_dialogue.txt")
summarzie_file = os.path.join(folder_path, f"{id}_sum.txt")
raw_text = chunk_text[id]
chunk_sum = []
unique_chunk_sum = []
# 给定summarzie_file = os.path.join(save_folder, f"{id}_sum.txt")

# 先检查这个文件是否存在

# 然后使用utf-8编码打开，检查每一行，如果strip后，行首是'-'，则把后面的字符串append到一个list chunk_sum中

# 请用python为我实现
if os.path.exists(summarzie_file):
    with open(summarzie_file, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip().startswith("-"):
                chunk_sum.append(line.strip()[1:].strip())
if os.path.exists(dialoge_file):
    with open(dialoge_file, encoding="utf-8") as f:
        dialogues = []
        for line in f:
            dialogue = json.loads(line)
            dialogues.append(dialogue)

unique_dialogue = []
for item in dialogues:
    if item not in unique_dialogue:
        unique_dialogue.append(item)
dia_texts = [data["dialogue"] for data in unique_dialogue]
for item in chunk_sum:
    if item not in unique_chunk_sum:
        unique_chunk_sum.append(item)

chunk_sum = unique_chunk_sum
dialogues = unique_dialogue
print(dialogues)
print(dia_texts)
print(chunk_sum)

[{'role': '烧盐柠檬', 'dialogue': '为你献上注定的败北', 'if_scene': False}, {'role': 'scene', 'dialogue': '提到“注定的败北”和“烧盐柠檬”', 'if_scene': True}, {'role': '老师', 'dialogue': '那种郁闷是什么，让老师来教·教·你', 'if_scene': False}, {'role': 'scene', 'dialogue': '提及一种郁闷感', 'if_scene': True}, {'role': 'scene', 'dialogue': '暗示老师将教授相关内容', 'if_scene': True}]
['为你献上注定的败北', '提到“注定的败北”和“烧盐柠檬”', '那种郁闷是什么，让老师来教·教·你', '提及一种郁闷感', '暗示老师将教授相关内容']
['提到“注定的败北”和“烧盐柠檬”', '提及一种郁闷感', '暗示老师将教授相关内容']


In [75]:
# 给定长文本raw_text。

# 使用换行符\n或者。 来对这个字符串进行切割，忽略掉strip之后是空的子字符串

# 将每一段话的起点位置存储在一个list of int , starts中

# 将每一段话的结束位置存储在一个list of int , ends中

# 并且将每一个子字符串的存储在一个list of str, lines中
def divide_raw2lines(raw_text):
    previous_str = ""
    starts = []
    ends = []
    lines = []
    for i in range(len(raw_text)):
        previous_str += raw_text[i]
        if raw_text[i] in ("\n", "。"):
            strip_str = previous_str.strip(' "“”\r\n')
            if len(strip_str) > 0:
                lines.append(strip_str)
                starts.append(i - len(strip_str))
                ends.append(i)
            previous_str = ""
        else:
            pass
    return lines, starts, ends


lines, starts, ends = divide_raw2lines(raw_text)

print(len(lines))

3


In [76]:
# 已知if '\u4e00' <= char <= '\u9fa5': 可以判断一个char是否是中文字

# 我希望实现一个函数，这个函数的输入是两个list of string, 长度为M的query 和 长度为N的datas

# 输出是一个M*N的numpy float数组 recalls

# 先计算freqs[m][n] 表示query的第m句中的每一个中文字，是否在datas[n]中是否出现，如果出现，则freqs[m][n]加一

# 然后计算recalls[m][n]是freqs[m][n]除掉 query[m]中所有中文字的个数


def compute_char_recall(query, datas):
    M = len(query)
    N = len(datas)

    freqs = np.zeros((M, N), dtype=int)

    for m in range(M):
        q_chars = set()
        for char in query[m]:
            if "\u4e00" <= char <= "\u9fa5":
                q_chars.add(char)

        for n in range(N):
            for char in q_chars:
                if char in datas[n]:
                    freqs[m][n] += 1

    query_chars_count = [
        len(set(char for char in sent if "\u4e00" <= char <= "\u9fa5"))
        for sent in query
    ]

    recalls = freqs / np.array(query_chars_count)[:, None]

    return recalls

In [77]:
# import copy
import numpy as np 
def summary2line(chunk_sum, lines):

    s = compute_char_recall(chunk_sum, lines)

    color_map = {}
    ans_Q = {}

    ans_div = {}

    flags = {}

    M = len(chunk_sum)
    N = len(lines)

    for n in range(0, N):
        if n == 0:
            ans_Q[(0, 0)] = s[0, 0]
            ans_div[(0, 0)] = []
        else:
            ans_Q[(0, n)] = ans_Q[(0, n - 1)] + s[0, n]
            ans_div[(0, n)] = []

    for m in range(1, M):
        ans_Q[(m, m)] = ans_Q[(m - 1, m - 1)] + s[m, m]
        ans_div[(m, m)] = ans_div[(m - 1, m - 1)].copy()
        ans_div[(m, m)].append(m)

    def find_Q(m, n):
        # print(m,n)

        if m < 0 or n < 0:
            print("error out bound", m, " ", n)
            return 0, []

        if (m, n) in ans_Q.keys():
            return ans_Q[(m, n)], ans_div[(m, n)]

        if (m, n) in color_map.keys():
            print("error repeated quest ", m, " ", n)
            return 0, []
        else:
            color_map[(m, n)] = 1

        current_div = []

        left, left_div = find_Q(m, n - 1)
        right, right_div = find_Q(m - 1, n - 1)

        if left > right:
            ans = left + s[m][n]
            flags[(m, n)] = False
            current_div = left_div

        else:
            ans = right + s[m][n]
            flags[(m, n)] = True
            current_div = right_div.copy()
            current_div.append(n - 1)

        # ans = max(  , ) + s[m][n]

        ans_Q[(m, n)] = ans
        ans_div[(m, n)] = current_div.copy()

        return ans, current_div

    # print(find_Q(0,5))
    # print(find_Q(M-1,N-1))

    score, divs = find_Q(M - 1, N - 1)
    divs.append(N - 1)

    return score, divs


score, divs = summary2line(chunk_sum, lines)
print(score, divs)

last = 0

# divs.append(N-1)

for i, div in enumerate(divs):
    print("\n###")
    print(chunk_sum[i])
    print("\n")

    for j in range(last, div):
        print(lines[j])
        last = div

1.0227272727272727 [1, 2, 2]

###
提到“注定的败北”和“烧盐柠檬”


为你献上注定的败北　烧盐柠檬

###
提及一种郁闷感


Intermission

###
暗示老师将教授相关内容




In [78]:
def dialogue2line(dia_texts, lines):
    s_dialogue = compute_char_recall(dia_texts, lines)

    M, N = s_dialogue.shape
    if M == 0 or N == 0:
        return []
    dp = np.zeros((M, N))
    dp[0] = s_dialogue[0]
    prev_indices = np.zeros((M, N), dtype=int)
    for i in range(1, M):
        for j in range(N):
            max_prev_index = np.argmax(dp[i - 1])
            dp[i][j] = dp[i - 1][max_prev_index] + s_dialogue[i][j]
            prev_indices[i][j] = max_prev_index

    max_end_index = np.argmax(dp[-1])
    sequence = []
    for i in range(M - 1, -1, -1):
        sequence.append(max_end_index)
        max_end_index = prev_indices[i][max_end_index]
    sequence.reverse()

    return sequence


seq = dialogue2line(dia_texts, lines)
print(seq)

for i, id in enumerate(seq):
    print("\n###")
    print(dia_texts[i])
    print(lines[id])
    print("\n")

    for j in range(last, div):
        print(lines[j])
        last = div

[0, 0, 2, 2, 2]

###
为你献上注定的败北
为你献上注定的败北　烧盐柠檬



###
提到“注定的败北”和“烧盐柠檬”
为你献上注定的败北　烧盐柠檬



###
那种郁闷是什么，让老师来教·教·你
那种郁闷是什么，让老师来教·教·你



###
提及一种郁闷感
那种郁闷是什么，让老师来教·教·你



###
暗示老师将教授相关内容
那种郁闷是什么，让老师来教·教·你




In [79]:
print(len(chunk_sum))
print(divs)
print(dia_texts)
print(seq)


# string_indices = [10, 11]
def jsonl_sorted(chunk_sum, divs, dia_texts, seq):

    combined_data = []
    combined_text = ""
    for index in sorted(seq + divs):
        # print(index)
        if index in seq:

            combined_data.append(
                {
                    "role": dialogues[seq.index(index)]["role"],
                    "text": dialogues[seq.index(index)]["dialogue"],
                    "if_scene": False,
                }
            )
            combined_text = (
                combined_text
                + dialogues[seq.index(index)]["role"]
                + ":"
                + dialogues[seq.index(index)]["dialogue"]
                + "\n"
            )
            seq[seq.index(index)] = -1
        if index in divs:
            combined_data.append(
                {
                    "role": "scene",
                    "text": chunk_sum[divs.index(index)],
                    "if_scene": True,
                }
            )
            combined_text = (
                combined_text + "scene" + ":" + chunk_sum[divs.index(index)] + "\n"
            )
            divs[divs.index(index)] = -1

    return combined_data, combined_text


combined_data, combined_text = jsonl_sorted(
    chunk_sum, divs.copy(), dia_texts, seq.copy()
)
print(combined_data)
# 打开文件，以写入模式（"w"）打开
with open(dialoge_file, "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
    for record in combined_data:
        # 将字典转换为JSON格式的字符串
        json_record = json.dumps(record, ensure_ascii=False)
        # 将转换后的JSON字符串写入文件，并添加换行符
        file.write(json_record + "\n")

3
[1, 2, 2]
['为你献上注定的败北', '提到“注定的败北”和“烧盐柠檬”', '那种郁闷是什么，让老师来教·教·你', '提及一种郁闷感', '暗示老师将教授相关内容']
[0, 0, 2, 2, 2]
[{'role': '烧盐柠檬', 'text': '为你献上注定的败北', 'if_scene': False}, {'role': 'scene', 'text': '提到“注定的败北”和“烧盐柠檬”', 'if_scene': False}, {'role': 'scene', 'text': '提到“注定的败北”和“烧盐柠檬”', 'if_scene': True}, {'role': '老师', 'text': '那种郁闷是什么，让老师来教·教·你', 'if_scene': False}, {'role': 'scene', 'text': '提及一种郁闷感', 'if_scene': True}, {'role': 'scene', 'text': '提及一种郁闷感', 'if_scene': False}, {'role': 'scene', 'text': '暗示老师将教授相关内容', 'if_scene': True}, {'role': 'scene', 'text': '暗示老师将教授相关内容', 'if_scene': False}]


In [80]:
os.system(f"rm -rf {save_folder_path}")
os.makedirs(save_folder_path, exist_ok=True)

In [81]:
final_jsonl = []
final_txt = ""

for i in tqdm(
    range(1, len(chunk_text)), desc="Processing", total=len(chunk_text) - 1, unit="item"
):

    try:
        # story_name_en = 'shediaoyingxiongzhuan'
        raw_text = chunk_text[i]

        import os

        save_folder = f"./extract/{story_name_en}"

        dialoge_file = os.path.join(save_folder, f"{i}_dialogue.txt")
        summarzie_file = os.path.join(save_folder, f"{i}_sum.txt")

        chunk_sum = []
        unique_chunk_sum = []
        if os.path.exists(summarzie_file):
            with open(summarzie_file, "r", encoding="utf-8") as f:
                for line in f:
                    if line.strip().startswith("-"):
                        chunk_sum.append(line.strip()[1:].strip())
        if os.path.exists(dialoge_file):
            with open(dialoge_file, encoding="utf-8") as f:
                dialogues = []
                for line in f:
                    dialogue = json.loads(line)
                    dialogues.append(dialogue)

        unique_dialogue = []
        for item in dialogues:
            if item not in unique_dialogue:
                unique_dialogue.append(item)
        dia_texts = [data["dialogue"] for data in unique_dialogue]
        for item in chunk_sum:
            if item not in unique_chunk_sum:
                unique_chunk_sum.append(item)

        chunk_sum = unique_chunk_sum
        dialogues = unique_dialogue
        lines, starts, ends = divide_raw2lines(raw_text)
        # print(chunk_sum)
        # print(lines)
        score, divs = summary2line(chunk_sum, lines)  # summary匹配
        seq = dialogue2line(dia_texts, lines)  # 对话匹配
        combined_data, combined_text = jsonl_sorted(
            chunk_sum, divs.copy(), dia_texts, seq.copy()
        )
        # 如果需要保存每个chunk的，在此处保存
        final_jsonl.append(combined_data)
        final_txt = final_txt + combined_text + "\n"
    except:
        print("第" + str(i) + "个chunk出错")
        pass
with open(save_jsonl_path, "w", encoding="utf-8") as file:
    # 遍历数据列表中的每个字典
    for record in final_jsonl:
        # 将字典转换为JSON格式的字符串
        json_record = json.dumps(record, ensure_ascii=False)
        # 将转换后的JSON字符串写入文件，并添加换行符
        file.write(json_record + "\n")
with open(save_txt_path, "w", encoding="utf-8") as file:
    file.write(final_txt)

Processing:   0%|          | 0/581 [00:00<?, ?item/s]C:\Users\boyu\AppData\Local\Temp\ipykernel_21548\281538791.py:34: RuntimeWarning: invalid value encountered in divide
  recalls = freqs / np.array(query_chars_count)[:, None]
Processing:   2%|▏         | 14/581 [00:00<00:04, 137.28item/s]

第2个chunk出错
第4个chunk出错


Processing:  34%|███▍      | 199/581 [00:01<00:03, 122.48item/s]

第188个chunk出错


Processing:  54%|█████▍    | 314/581 [00:02<00:02, 132.84item/s]

第291个chunk出错


Processing:  71%|███████   | 410/581 [00:03<00:01, 126.78item/s]

第390个chunk出错


Processing: 100%|██████████| 581/581 [00:04<00:00, 121.24item/s]


# 4.小说chatbot抽取

In [82]:
# 参数设置

# 支持跨越多少行寻找目标角色，也即控制段内行间距不超过该值
max_find_lines = 10

max_token_num = 500

# target_role支持 空字符串(默认前三个)或者List of string 如果出错默认保存第一个
target_role = ["八奈见", "我"]

# 输入文件路径
input_jsonl_name = (
    f"./reorganized/{novel_name}/reorganized_{novel_name}.jsonl"
)

# 保存路径
savepath = f"./role/{novel_name}"
os.system(f"rm -rf {savepath}")
os.makedirs(savepath, exist_ok=True)
# zip_path = '/content/linghuchong_text.zip'

In [83]:
# input_name = '/content/reorganized_xiaoaojianghu.jsonl'
# 用utf8编码为我读取jsonl文件，并且把每一行解析成一个list of json, data
# 请用python为我实现

# # 输入文件路径
# input_name = '/content/shediaoyingxiongzhuan.jsonl'

enc = tiktoken.get_encoding("cl100k_base")

data_in_chunk = []
with open(input_jsonl_name, encoding="utf8") as f:
    for line in f:
        data_in_chunk.append(json.loads(line))

data = []

for chunk in data_in_chunk:
    for d in chunk:
        data.append(d)

print(len(data))
for i, d in enumerate(data):
    if d["role"] == "scene":
        first_scene_id = i
        break
print(first_scene_id)

17771
1


In [84]:
def output_scene_chat_id(data, target_role_single):

    chat_ids = []

    # 先寻找所有出现角色的节点
    for i, d in enumerate(data):
        if d["role"] == target_role_single:
            chat_ids.append(i)

    previous_scene_ids = []

    # 对于每一个chat_ids，向前寻找scene的节点
    for chat_id in chat_ids:
        ans = first_scene_id
        for j in range(chat_id, first_scene_id, -1):
            if data[j]["role"] == "scene":
                ans = j
                break
        previous_scene_ids.append(ans)
    return chat_ids, previous_scene_ids

In [85]:
def divide_chats2chunks(chat_ids, previous_scene_ids):
    chat_ids_in_chunk = []
    current_chunk = []

    for chat_id in chat_ids:
        if not current_chunk:
            current_chunk.append(chat_id)
            continue

        if chat_id - current_chunk[-1] <= max_find_lines:
            current_chunk.append(chat_id)
        else:
            chat_ids_in_chunk.append(current_chunk)
            current_chunk = [chat_id]

    if current_chunk:
        chat_ids_in_chunk.append(current_chunk)

    # print(chat_ids_in_chunk[0])

    chat_id2previous_scene_id = {}

    for previous, chat_id in zip(previous_scene_ids, chat_ids):
        chat_id2previous_scene_id[chat_id] = previous
        if previous > 0:
            if data[previous - 1]["role"] != target_role:
                chat_id2previous_scene_id[chat_id] -= 1
    return chat_ids_in_chunk, chat_id2previous_scene_id


# chat_ids的分块， chat_id对应的旁白id

In [86]:
def count_token(my_str):
    return len(enc.encode(my_str))


def data2str(data):
    role = data["role"]
    if role in ["旁白", "", "scene", "Scene", "narrator", "Narrator"]:
        return "scene:" + data["text"]
    else:
        return role + ":「" + data["text"] + "」"

In [87]:
def id2texts(data, chat_ids_in_chunk, chat_id2previous_scene_id):
    line_token = [count_token(data2str(d)) for d in data]
    from ast import Break

    final_chunks = []

    print_count = 0

    appended_key = []

    for chunk in chat_ids_in_chunk:
        N = len(chunk)

        current_i = 0

        while current_i < N - 1:

            consider_chat_id = chunk[current_i]

            previous_scene_id = chat_id2previous_scene_id[consider_chat_id]

            # 保底
            withdraw_start = previous_scene_id
            withdraw_end = consider_chat_id

            current_count = sum(line_token[previous_scene_id : consider_chat_id + 1])
            while current_count < max_token_num and current_i < N - 1:
                consider_end = chunk[current_i + 1]
                consider_count = sum(line_token[previous_scene_id : consider_end + 1])
                if consider_count < max_token_num:
                    current_count = consider_count
                    withdraw_start = previous_scene_id
                    withdraw_end = consider_end
                    current_i += 1
                else:
                    break

            # print_count += 1

            # print(withdraw_start, end = ' ')
            # if print_count % 5 == 0:
            #     print()

            if withdraw_end + 1 not in appended_key:
                appended_key.append(withdraw_end + 1)
                chunk_str = ""
                for i in range(withdraw_start, withdraw_end + 1):
                    chunk_str += data2str(data[i]) + "\n"

                final_chunks.append(chunk_str)

            current_i += 1
    return appended_key, final_chunks

In [88]:
import zipfile
def save_chunk2zip(savepath, save_title, final_chunks):
    os.makedirs(savepath, exist_ok=True)
    for i in range(0, len(final_chunks)):
        my_str = final_chunks[i]
        with open(savepath + f"/text_{i}.txt", "w", encoding="utf-8") as f:
            f.write(my_str)
    zip_path = "./role/" + save_title + "_text.zip"
    with zipfile.ZipFile(zip_path, "w") as zipf:
        for filename in os.listdir(savepath):
            zipf.write(savepath + "/" + filename)

    print("Zipped folder saved to", zip_path)

In [89]:
for role_cur_name in target_role:
    chat_ids, previous_scene_ids = output_scene_chat_id(data, role_cur_name)
    chat_ids_in_chunk, chat_id2previous_scene_id = divide_chats2chunks(
        chat_ids, previous_scene_ids
    )
    appended_key, final_chunks = id2texts(
        data, chat_ids_in_chunk, chat_id2previous_scene_id
    )
    save_chunk2zip(
        savepath + "/" + role_cur_name, role_cur_name, final_chunks
    )  # 如果你想修改保存的zip名称，请修改本函数的第二个参数save_title

Zipped folder saved to ./role/八奈见_text.zip
Zipped folder saved to ./role/我_text.zip


# 5.抽取embedding
获得角色的jsonl资料后需要进行embedding构建支持RAG的知识库，这一步移到test文件里进行了